Example notebook for defining networks and training expert models.

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

import keras
import tensorflow as tf
from keras import layers
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Dropout
from keras import initializers
from keras.models import load_model
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.utils.class_weight import compute_class_weight
from keras.metrics import Precision, Recall
import matplotlib.pyplot as plt
import numpy as np
from keras.optimizers.schedules import ExponentialDecay
from keras.utils import to_categorical
import scipy

import pandas as pd
import os
import sys
import pickle
import random
import time
import itertools
from concurrent.futures import ProcessPoolExecutor

#Ensuring script directory is correct
script_dir = os.path.dirname(os.path.abspath(sys.argv[0]))
os.chdir(script_dir)

Necessary Functions:

In [2]:
def set_seeds(seed):
    np.random.seed(seed)
    random.seed(seed)
    tf.random.set_seed(seed)

def build_model(input_dim, layer_neurons, activation='tanh'):
    #Setting up generic model inputs based on layers and neurons
    model = keras.Sequential()
    model.add(layers.Dense(layer_neurons[0], activation=activation, input_dim=input_dim))
    for i in range(1, len(layer_neurons) - 1):
        model.add(layers.Dense(layer_neurons[i], activation=activation))
    model.add(layers.Dense(layer_neurons[-1], activation='softplus'))
    return model

def relative_mse(y_true, y_pred):
    #Defining MSE loss function
    return tf.reduce_mean(tf.square((y_pred - y_true) / (y_true + 1e-6)))

def evaluate_metrics(y_test, y_pred, X_test=None, water=False):
    #Building evaluation metrics
    mape = (1/len(y_test))*np.sum(abs(np.array(y_test)-np.array(y_pred))/(np.array(y_test)))
    nmae = (1/np.mean(y_test))*(1/len(y_test))*np.sum(abs(np.array(y_test)-np.array(y_pred)))
    rmse = np.sqrt((1/len(y_test))*np.sum((np.array(y_test)-np.array(y_pred))**2))
    if water:
        #Comment in/out version for ammonium cases
        print('NH4 != 0')
        unscale_factor = (X_test['NH4+'] + X_test['NH4+']*(X_test['NA+']+X_test['SO42-'] + X_test['NO3-'] + X_test['CL-'])) * (X_test['RH'] / (1 - X_test['RH']))
   
        y_test_unscaled = np.array(y_test).flatten() * np.array(unscale_factor).flatten()
        y_pred_unscaled = np.array(y_pred).flatten() * np.array(unscale_factor).flatten()
        mass_error =  (1/(len(y_test_unscaled))) * np.sum((abs((y_test_unscaled*18)-(np.asarray(y_pred_unscaled).reshape(-1)*18)))/((X_test['NH4+']*18)+ X_test['NH4+']*((X_test['NA+']*23) + (X_test['SO42-']*96) + (X_test['NO3-']*62) + (X_test['CL-']*35.5)) + (y_test_unscaled * 18)))

        # print('NH4 = 0')
        # unscale_factor = (X_test['NA+'] + X_test['NA+']*(X_test['SO42-'] + X_test['NO3-'] + X_test['CL-'])) * (X_test['RH'] / (1 - X_test['RH']))
   
        # y_test_unscaled = np.array(y_test).flatten() * np.array(unscale_factor).flatten()
        # y_pred_unscaled = np.array(y_pred).flatten() * np.array(unscale_factor).flatten()
        # mass_error =  (1/(len(y_test_unscaled))) * np.sum((abs((y_test_unscaled*18)-(np.asarray(y_pred_unscaled).reshape(-1)*18)))/((X_test['NA+']*23) + X_test['NA+']*((X_test['SO42-']*96) + (X_test['NO3-']*62) + (X_test['CL-']*35.5)) + (y_test_unscaled * 18)))
        return {'RMSE': rmse, 'NMAE': nmae, 'MAPE': mape, 'mass_error': mass_error}
    else:
        return {'RMSE': rmse, 'NMAE': nmae, 'MAPE': mape}

def gaussian_loglik(y_true, y_pred):
    #Gaussian loglikelihood
    y_true = np.asarray(y_true).ravel()
    y_pred = np.asarray(y_pred).ravel()
    n = len(y_true)
    resid = y_true - y_pred
    sigma2 = np.mean(resid**2)
    return -0.5 * n * (np.log(2 * np.pi * sigma2) + 1)

def logmeanexp(x):
    xmax = np.max(x)
    return xmax + np.log(np.mean(np.exp(x - xmax)))


def train_model(arch, seed, X_trainval, y_trainval, X_test, y_test, batch_size, epochs, patience, 
                lr_exp_scheduler, filepath, lr_callback = None, water=False):
    tf.keras.backend.clear_session()
    print(f'Training architecture: {arch}')
    set_seeds(seed)
    print(f'\tSeed: {seed}')
    #Ammonium present water case
    predictors = ['TEMP', 'RH', 'NA+', 'SO42-', 'NO3-', 'CL-']
    #Ammonium absent case
    #predictors = ['TEMP', 'RH', 'SO42-', 'NO3-', 'CL-']

    #Creating train/val spit from the x_trainval
    X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, test_size = 0.25, shuffle=True, random_state = seed)

    #Scaling training, testing data
    X_val_copy = X_val.copy()
    scalers = {}
    for col in ['TEMP', 'RH']:
        mean = X_train[col].mean()
        std = X_train[col].std(ddof=0)
        X_train[col] = (X_train[col] - mean) / std
        X_val[col] = (X_val[col] - mean) / std
        scalers[col] = (mean, std)
    
    X_test_copy = X_test.copy()
    #Scaling global test data
    X_test_scaled = X_test.copy()
    for col in ['TEMP', 'RH']:
        mean, std = scalers[col]
        X_test_scaled[col] = (X_test_scaled[col] - mean) / std
    print('\t\tConstructing model...')
    #model construction

    if water:
        #Scaling water
        # y_train['water_content'] = y_train['water_content']/((X_train['SO42-']+X_train['NO3-']+X_train['CL-']) * ((X_train['RH'])/(1-X_train['RH'])))
        # y_val['water_content'] = y_val['water_content']/((X_val['SO42-']+X_val['NO3-']+X_val['CL-']) * ((X_val['RH'])/(1-X_val['RH'])))
        # y_test['water_content'] = y_test['water_content']/((X_test_scaled['SO42-']+X_test_scaled['NO3-']+X_test_scaled['CL-']) * ((X_test_scaled['RH'])/(1-X_test_scaled['RH'])))
        #model construction
        model = build_model(input_dim = len(predictors), layer_neurons=arch)
        optimizer = keras.optimizers.legacy.Adam(learning_rate=0.001)
        model.compile(optimizer = optimizer, loss = tf.keras.losses.Huber(), metrics=['MeanSquaredError'])
    else:
        model = build_model(input_dim = len(predictors), layer_neurons=arch)
        optimizer = keras.optimizers.legacy.Adam(learning_rate=lr_exp_scheduler)
        model.compile(optimizer = optimizer, loss = relative_mse, metrics=['MeanSquaredError'])


    #Callbacks
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=patience,
        min_delta=0.00005,
        mode='min',
        restore_best_weights=True
    )

    if lr_callback:
        lr_cb = tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)

    print('\t\tTraining...')
    #Training
    start = time.time()
    if water:
        history = model.fit(X_train.loc[:,predictors], y_train, validation_data=(X_val.loc[:,predictors], y_val), batch_size=batch_size, epochs=epochs, shuffle=True, callbacks=[early_stopping, lr_cb], verbose=0)
    else:
        history = model.fit(X_train, y_train, validation_data=(X_val, y_val), batch_size=batch_size, epochs=epochs, shuffle=True, callbacks=[early_stopping], verbose=0)
    end = time.time()
    elapsed = end-start
    print(f'\t\tTraining time: {elapsed} seconds')
    #Final validation evaluation
    print('\t\tEvaluating validation set...')
    y_val_pred = model.predict(X_val.loc[:,predictors])
    if water:
        val_metrics = evaluate_metrics(y_val, y_val_pred, X_val_copy, water = True)
    else:
        val_metrics = evaluate_metrics(y_val, y_val_pred)
    print(f'\t\t{val_metrics}')
    #Final evaluation on test set
    print('\t\t Evaluating test set')
    y_test_pred = model.predict(X_test_scaled.loc[:,predictors])
    if water:
        test_metrics = evaluate_metrics(y_test, y_test_pred, X_test_copy, water=True)
    else:
        test_metrics = evaluate_metrics(y_test, y_test_pred)
    print(f'\t\t{test_metrics}')
    #Extract model complexity
    total_params = model.count_params()
    #Calculate log likelihood
    test_loglike = gaussian_loglik(y_test.values, y_test_pred)

    results = {
        'n_layers': len(arch),
        'neurons': arch,
        'seed': seed,
        'params': total_params,
        'val_RMSE': val_metrics['RMSE'],
        'val_MAPE': val_metrics['MAPE'],
        'val_NMAE': val_metrics['NMAE'],
        'test_RMSE': test_metrics['RMSE'],
        'test_MAPE': test_metrics['MAPE'],
        'test_NMAE': test_metrics['NMAE'],
        'test_loglike': test_loglike,
        'train_time':elapsed,
        'epochs_trained':len(history.history['loss']),
        'training_history': history.history
    }
    if water:
        results['val_mass_error'] = val_metrics['mass_error']
        results['test_mass_error'] = test_metrics['mass_error']

    model_name = f"arch_{'_'.join(map(str, arch))}_seed_{seed}"
    save_path = os.path.join(script_dir, filepath, model_name)
    os.makedirs(save_path, exist_ok=True)

    # 1. Save the full model (Best Practice)
    model.save(f"{save_path}/full_model.keras")

    # 2. Save weights and JSON separately (If you specifically need them)
    model.save_weights(f"{save_path}/weights.weights.h5")
    with open(f"{save_path}/metadata.json", "w") as f:
        f.write(model.to_json())

    return results

Loading and transforming data:

In [3]:
#Loading raw data, editing column names and binning liquid + mixed into one class
df = pd.read_parquet(f'/Users/jeremyelvander/Desktop/AQRC Research/ml_files/data_procurement/eaim_training_final.parquet')
df = df.drop(columns=['n_H2O_g'])
df = df.rename(columns={'n_H2O_aq':'water_content'})
df['phase'] = df['phase'].replace(2, 0)
df['phase'] = df['phase'].replace(3, 0)

#Ensuring correct ratio scaling
with open('/Users/jeremyelvander/Desktop/AQRC Research/ml_files/new_models/solid_amm_nit_nonzero.pkl', 'rb') as file:
    load_model = pickle.load(file)
b = load_model.coef_[0]
a = np.exp(load_model.intercept_)

df['amm_nit_ratio'] = df['amm_nit']/(a * np.exp(b * (1/df['TEMP'])))

#Ensuring correct ratio scaling
with open('/Users/jeremyelvander/Desktop/AQRC Research/ml_files/new_models/solid_amm_chl_nonzero_NEW.pkl', 'rb') as file:
    load_model = pickle.load(file)
b = load_model.coef_[0]
a = np.exp(load_model.intercept_)

df['amm_chl_ratio'] = df['amm_chl']/(a * np.exp(b * (1/df['TEMP'])))

df = df[df['amm_nit_ratio'] <= 1.]
df = df[df['amm_chl_ratio'] <= 1.]


#Building dataset for NH4 = 0 Case
df_nh4_zero = df[df['NH4+'] == 0].copy()
cols_zero = ['SO42-', 'NO3-', 'CL-']
df_nh4_zero['water_content'] = df_nh4_zero['water_content']/((df_nh4_zero['NA+']+df_nh4_zero['SO42-']+df_nh4_zero['NO3-']+df_nh4_zero['CL-']) * ((df_nh4_zero['RH'])/(1-df_nh4_zero['RH'])))
df_nh4_zero.loc[:,cols_zero] = df_nh4_zero[cols_zero].div(df_nh4_zero['NA+'], axis=0)
df_na = df_nh4_zero.loc[:,'NA+']
#df_nh4_zero.drop(columns=['NH4+', 'NA+'], inplace=True)

#Building dataset for NH4 =/= 0 case
df_nh4_nonzero = df[df['NH4+'] != 0].copy()
cols_nonzero = ['NA+', 'SO42-', 'NO3-', 'CL-']
df_nh4_nonzero['water_content'] = df_nh4_nonzero['water_content']/((df_nh4_nonzero['NH4+']+df_nh4_nonzero['NA+']+df_nh4_nonzero['SO42-']+df_nh4_nonzero['NO3-']+df_nh4_nonzero['CL-']) * ((df_nh4_nonzero['RH'])/(1-df_nh4_nonzero['RH'])))
df_nh4_nonzero.loc[:,cols_nonzero] = df_nh4_nonzero[cols_nonzero].div(df_nh4_nonzero['NH4+'], axis=0)
df_nh4 = df_nh4_nonzero.loc[:,'NH4+']
#df_nh4_nonzero.drop(columns=['NH4+'], inplace=True)

df_nh4_zero.reset_index(drop=True, inplace=True)
df_nh4_nonzero.reset_index(drop=True, inplace=True)

NH4+ =/= 0 Phase Classifier

In [ ]:
predictors = ['TEMP', 'RH', 'NA+', 'SO42-', 'NO3-', 'CL-']

#Setting up training/validation sets
X = df_nh4_nonzero.loc[:,predictors]
y = df_nh4_nonzero.loc[:,['phase']]

#Normalizing temperature and relative humidity
scalers = {}
for col in ['TEMP', 'RH']:
    mean = X[col].mean()
    std = X[col].std(ddof=0)
    X[col] = (X[col] - mean) / std
    scalers[col] = (mean, std)


#split for train / test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle=True)

#split for val
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25)

y_train = y_train.astype(int)
y_val = y_val.astype(int)
y_test = y_test.astype(int)

#Training model
phase_classifier = Sequential()
phase_classifier.add(Dense(input_dim = 6, units=32, activation='tanh')) # 16
phase_classifier.add(Dense(units=16,activation='tanh'))
phase_classifier.add(Dense(units=8,activation='tanh')) #6
phase_classifier.add(Dense(units=4,activation='tanh')) #new
phase_classifier.add(Dense(units=1, activation='sigmoid'))

lr_exp_scheduler = ExponentialDecay(
    initial_learning_rate = 0.001,
    decay_steps=81973,
    decay_rate=0.96,
    staircase=False
)

def focal_loss(gamma=2., alpha=.6):
    def loss(y_true, y_pred):
        y_pred = tf.clip_by_value(y_pred, 1e-7, 1. - 1e-7)
        pt = tf.where(tf.equal(y_true, 1), y_pred, 1 - y_pred)
        return -tf.reduce_mean(alpha * tf.pow(1. - pt, gamma) * tf.math.log(pt))
    return loss

optimizer = keras.optimizers.legacy.Adam(learning_rate=lr_exp_scheduler)
phase_classifier.compile(optimizer = optimizer, loss = focal_loss(), metrics=['binary_crossentropy', 'accuracy', Precision(), Recall()])

#Adding early stopping to prevent overfitting
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy',
    patience=40,
    min_delta=0.0005,
    mode='max',
    restore_best_weights=True
)

class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train['phase'].values)
class_weight_dict = dict(enumerate(class_weights))

history = phase_classifier.fit(X_train, y_train, validation_data=(X_val, y_val), batch_size=512, epochs=1000, shuffle=True, 
class_weight=class_weight_dict, callbacks=[early_stopping])

# phase_classifier.save('/Users/jeremyelvander/Desktop/AQRC Research/ml_files/new_models/phase_classifier_nonzero.keras')

done


KeyboardInterrupt: 

NH4+ =/= 0, Liquid/Mix, Ammonium Nitrate Network

In [ ]:
nonzero_liqmix_pp = df_nh4_nonzero[df_nh4_nonzero['phase']==0][df_nh4_nonzero['amm_nit']!=0].copy()

# plt.hist(nonzero_liqmix_pp['amm_nit'], bins=100)
# plt.xlabel('Ammonium Nitrate (Units)')
# plt.ylabel('Count')
# plt.title('Ammonium Nitrate (Raw) - NH4 =/= 0, Liquid/Mix Case')
# plt.grid(True)
# plt.show()

#Loading amm_nit solid case exponential model for data normalization
with open('/Users/jeremyelvander/Desktop/AQRC Research/ml_files/new_models/solid_amm_nit_nonzero.pkl', 'rb') as file:
    load_model = pickle.load(file)
b = load_model.coef_[0]
a = np.exp(load_model.intercept_)
#Transforming data
nonzero_liqmix_pp['amm_nit'] = nonzero_liqmix_pp['amm_nit']/(a * np.exp(b * (1/nonzero_liqmix_pp['TEMP'])))

# plt.hist(nonzero_liqmix_pp['amm_nit'], bins=100)
# plt.xlabel('Ammonium Nitrate (Units)')
# plt.ylabel('Count')
# plt.title('Ammonium Nitrate (Scaled) - NH4 =/= 0, Liquid/Mix Case')
# plt.grid(True)
# plt.show()

predictors = ['TEMP', 'RH', 'NA+', 'SO42-', 'NO3-', 'CL-']
X = nonzero_liqmix_pp.loc[:,predictors].copy()
y = nonzero_liqmix_pp.loc[:,['amm_nit']].copy()


X_trainval, X_test, y_trainval, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=4722)


if __name__ == "__main__":
    #Load and Process Data
    filepath = 'NH4_nonzero/amm_nit/'

    architectures = [
        (4, 1),
        (8, 4, 1),
        (16, 8, 1),
        (20, 10, 5, 1),
        (32, 16, 8, 1),
        (64, 32, 16, 1),
        (128, 64, 32, 16, 1)
    ]
    seeds = (1, 2, 3, 4, 5, 6, 7, 8, 9, 10)
    results = []

    batch_size = 512
    epochs = 500
    patience = 40
    lr = 0.0005

    #Setup scheduler
    decay_steps = int((len(X_trainval)*0.75)/batch_size * 25)
    lr_exp_scheduler = ExponentialDecay(
        initial_learning_rate = lr,
        decay_steps=decay_steps,
        decay_rate=0.96,
        staircase=False
    )


    #Run with ProcessPoolExecutor
    print("Starting Training across multiple processes...")
    with ProcessPoolExecutor(max_workers=4) as executor:
        futures = [
            executor.submit(train_model, arch, seed, X_trainval, y_trainval, X_test, y_test, 
                            batch_size, epochs, patience, lr_exp_scheduler, filepath, water=False)
            for arch, seed in itertools.product(architectures, seeds)
        ]
        results = [f.result() for f in futures]

    #Save Final Results
    df_results = pd.DataFrame(results)

    df_results.to_csv('/Users/jeremyelvander/Desktop/AQRC Research/ml_files/model_training/NH4_nonzero/amm_nit/nz_lm_amm_nit_nn.csv')
    print("All models trained and results saved.")

/var/folders/5_/y_2p6lf540s0fskbmjbhg1qc0000gn/T/ipykernel_27283/1718721625.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  nonzero_liqmix_pp = df_nh4_nonzero[df_nh4_nonzero['phase']==0][df_nh4_nonzero['amm_nit']!=0].copy()


done


KeyboardInterrupt: 

NH4+ =/= 0, Liquid/Mix, Ammonium Chloride

In [ ]:
#NH4 =/= 0, Liq/Mix, Amm_Chl Network 
nonzero_liqmix_pp = df_nh4_nonzero[df_nh4_nonzero['phase']==0][df_nh4_nonzero['amm_chl']!=0].copy()

# plt.hist(nonzero_liqmix_pp['amm_chl'], bins=100)
# plt.xlabel('Ammonium Chloride (Units)')
# plt.ylabel('Count')
# plt.title('Ammonium Chloride (Raw) - NH4 =/= 0, Liquid/Mix Case')
# plt.grid(True)
# plt.show()

#Loading amm_nit solid case exponential model for data normalization
with open('/Users/jeremyelvander/Desktop/AQRC Research/ml_files/new_models/solid_amm_chl_nonzero_NEW.pkl', 'rb') as file:
    load_model = pickle.load(file)
b = load_model.coef_[0]
a = np.exp(load_model.intercept_)
#Transforming data
nonzero_liqmix_pp['amm_chl'] = nonzero_liqmix_pp['amm_chl']/(a * np.exp(b * (1/nonzero_liqmix_pp['TEMP'])))

# plt.hist(nonzero_liqmix_pp['amm_chl'], bins=100)
# plt.xlabel('Ammonium Chloride (Units)')
# plt.ylabel('Count')
# plt.title('Ammonium Chloride (Raw) - NH4 =/= 0, Liquid/Mix Case')
# plt.grid(True)
# plt.show()

predictors = ['TEMP', 'RH', 'NA+', 'SO42-', 'NO3-', 'CL-']
X = nonzero_liqmix_pp.loc[:,predictors].copy()
y = nonzero_liqmix_pp.loc[:,['amm_chl']].copy()


X_trainval, X_test, y_trainval, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=4722)


if __name__ == "__main__":
    #Load and process data and parameters
    filepath = 'NH4_nonzero/amm_chl/'
    architectures = [
    (4, 1),
    (8, 4, 1),
    (16, 8, 1),
    (20, 10, 5, 1),
    (32, 16, 8, 1),
    (64, 32, 16, 1),
    (128, 64, 32, 16, 1)
    ]
    seeds = (1, 2, 3, 4, 5, 6, 7, 8, 9, 10)
    results = []

    batch_size = 512
    epochs = 500
    patience = 40
    lr = 0.0005
    #Setup scheduler
    decay_steps = int((len(X_trainval)*0.75)/batch_size * 25)
    lr_exp_scheduler = ExponentialDecay(
        initial_learning_rate = lr,
        decay_steps=decay_steps,
        decay_rate=0.96,
        staircase=False
    )
    #Run with ProcessPool
    print("Starting Training across multiple processes...")
    with ProcessPoolExecutor(max_workers=4) as executor:
        futures = [
            executor.submit(train_model, arch, seed, X_trainval, y_trainval, X_test, y_test, 
                            batch_size, epochs, patience, lr_exp_scheduler, filepath, water=False)
            for arch, seed in itertools.product(architectures, seeds)
        ]
        results = [f.result() for f in futures]


    df_results = pd.DataFrame(results)

    df_results.to_csv('/Users/jeremyelvander/Desktop/AQRC Research/ml_files/model_training/NH4_nonzero/amm_chl/nz_lm_amm_chl_nn.csv')

/var/folders/5_/y_2p6lf540s0fskbmjbhg1qc0000gn/T/ipykernel_27283/367297124.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  nonzero_liqmix_pp = df_nh4_nonzero[df_nh4_nonzero['phase']==0][df_nh4_nonzero['amm_chl']!=0].copy()


done


KeyboardInterrupt: 

NH4+ =/= 0, Liquid/Mix, Water Content

In [ ]:
nonzero_liquid_mix = df_nh4_nonzero[df_nh4_nonzero['phase']==0].copy()
nonzero_liquid_mix = nonzero_liquid_mix[nonzero_liquid_mix['water_content'] > 0.009]

#Building training/validation sets
predictors = ['TEMP', 'RH', 'NA+', 'SO42-', 'NO3-', 'CL-']
X = nonzero_liquid_mix#.loc[:,predictors]
y = nonzero_liquid_mix.loc[:,['water_content']]


X_trainval, X_test, y_trainval, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=4722)


if __name__ == "__main__":
    #Setting up parameters and loading data
    filepath = 'NH4_nonzero/water_content/'

    architectures = [
    (4, 1),
    (8, 4, 1),
    (16, 8, 1),
    (20, 10, 5, 1),
    (32, 16, 8, 1),
    (64, 32, 16, 1),
    (128, 64, 32, 16, 1)
    ]
    seeds = (1, 2, 3, 4, 5, 6, 7, 8, 9, 10)
    results = []

    batch_size = 1024
    epochs = 500
    patience = 40
    #Establishing lr callback
    lr_callback = True

    lr = 0.0005

    decay_steps = int((len(X_trainval)*0.75)/batch_size * 25)
    lr_exp_scheduler = ExponentialDecay(
        initial_learning_rate = lr,
        decay_steps=decay_steps,
        decay_rate=0.96,
        staircase=False
    )

    #Running with ProcessPool
    with ProcessPoolExecutor(max_workers=4) as executor:
        futures=[
            executor.submit(train_model, arch, seed, X_trainval, y_trainval, X_test, y_test, batch_size, epochs, patience, 
                    lr_exp_scheduler, filepath, lr_callback=lr_callback, water=True)
            for arch, seed in itertools.product(architectures, seeds)
        ]
        results = [f.result() for f in futures]

    df_results = pd.DataFrame(results)

    df_results.to_csv('/Users/jeremyelvander/Desktop/AQRC Research/ml_files/model_training/NH4_nonzero/water_content/nz_lm_water_content_nn.csv')

done


KeyboardInterrupt: 

NH4+ = 0, Phase Classifier

In [ ]:
predictors = ['TEMP', 'RH', 'SO42-', 'NO3-', 'CL-']
X = df_nh4_zero.loc[:,predictors]
y = df_nh4_zero.loc[:,['phase']]

scalers = {}
for col in ['TEMP', 'RH']:
    mean = X[col].mean()
    std = X[col].std(ddof=0)
    X[col] = (X[col] - mean) / std
    scalers[col] = (mean, std)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle=True)

#split for val
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25)

y_train = y_train.astype(int)
y_val = y_val.astype(int)
y_test = y_test.astype(int)

phase_classifier = Sequential()
phase_classifier.add(Dense(input_dim = 5, units=8, activation='tanh'))
phase_classifier.add(Dense(units=3,activation='tanh'))
phase_classifier.add(Dense(units=3,activation='tanh'))
phase_classifier.add(Dense(units=1, activation='sigmoid'))

lr_exp_scheduler = ExponentialDecay(
    initial_learning_rate = 0.005,
    decay_steps=5156,
    decay_rate=0.96,
    staircase=False
)


optimizer = keras.optimizers.legacy.Adam(learning_rate=lr_exp_scheduler)
phase_classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics=['binary_crossentropy', 'accuracy', Precision(), Recall()])

#Adding early stopping to prevent overfitting
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy',
    patience=40,
    min_delta=0.0005,
    mode='max',
    restore_best_weights=True
)

class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train['phase'].values)
class_weight_dict = dict(enumerate(class_weights))

history = phase_classifier.fit(X_train, y_train, validation_data=(X_val, y_val), batch_size=64, epochs=1000, shuffle=True, 
class_weight=class_weight_dict, callbacks=[early_stopping])

# phase_classifier.save('/Users/jeremyelvander/Desktop/AQRC Research/ml_files/new_models/phase_classifier_zero.keras')

done


KeyboardInterrupt: 

NH4+ = 0, Liquid/Mix, Water Content

In [ ]:
zero_liquid_mix = df_nh4_zero[df_nh4_zero['phase']==0].copy()


predictors = ['TEMP', 'RH', 'SO42-', 'NO3-', 'CL-']

#Setting up training and test sets
X = zero_liquid_mix
y = zero_liquid_mix.loc[:,['water_content']]

X_trainval, X_test, y_trainval, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=4722)



if __name__ == "__main__":
    #Establishing parameter input
    filepath = 'NH4_zero/water_content/'

    architectures = [
        (4, 1),
        (8, 4, 1),
        (16, 8, 1),
        (20, 10, 5, 1),
        (32, 16, 8, 1),
        (64, 32, 16, 1),
        (128, 64, 32, 16, 1)
    ]
    seeds = (1, 2, 3, 4, 5, 6, 7, 8, 9, 10)
    results = []

    batch_size = 1024
    epochs = 500
    patience = 40

    lr_callback = True

    lr = 0.0005

    decay_steps = int((len(X_trainval)*0.75)/batch_size * 25)
    lr_exp_scheduler = ExponentialDecay(
        initial_learning_rate = lr,
        decay_steps=decay_steps,
        decay_rate=0.96,
        staircase=False
    )

    #Running with processPool
    with ProcessPoolExecutor(max_workers=4) as executor:
        futures=[
            executor.submit(train_model, arch, seed, X_trainval, y_trainval, X_test, y_test, batch_size, epochs, patience, 
                    lr_exp_scheduler, filepath, lr_callback, water=True)
            for arch, seed in itertools.product(architectures, seeds)
        ]
        results = [f.result() for f in futures]


    df_results = pd.DataFrame(results)
    df_results.to_csv('/Users/jeremyelvander/Desktop/AQRC Research/ml_files/model_training/NH4_zero/water_content/z_lm_water_content_nn.csv')

done


KeyboardInterrupt: 